In [84]:
import pandas as pd
import urllib.parse
from random import shuffle
from my_rymscraper.rymscraper import rymscraper, RymUrl
network = rymscraper.RymNetwork()

In [85]:
mt = pd.read_csv("metacritic_data.csv")

In [86]:
# mt['tag'] = mt["Singer"] + " - " + mt["Album_Name"];
lower_alnum = lambda s: "".join(x for x in s.lower() if x.isalnum())
mt["id"] = mt["Singer"].apply(lower_alnum) + mt["Release_Year"].apply(str) + mt["Album_Name"].apply(lambda s: lower_alnum(s)[:3] + lower_alnum(s)[-3:])
tags = list(zip(mt["id"], mt["Singer"], mt["Album_Name"]))
# tags = ["Common - A Beautiful Revolution, Vol. 2"]
cols = ['Id', 'Name', 'Artist', 'Released', 'RYM Rating', 'Genres', 'Descriptors',
       'Language', 'Track listing', 'Colorscheme', 'Languages', 'Ranked']
df = pd.DataFrame(columns=cols)

In [87]:
lower_alnum_dash = lambda s: "".join(x for x in s.replace(" ", "-").lower() if (x.isalnum() or x == "-" ))
# tags = [('divmo','Maya Hawke', 'MOSS')]

max_iter = 1 # one iteration is enough for most cases, raise it if errors occur
iter = 0
while iter < max_iter:
  new_tags = []
  for uid, singer, album in tags:
    try:
      # Try url first
      url = "https://rateyourmusic.com/release/album/%s/%s/" % (lower_alnum_dash(singer), lower_alnum_dash(album))
      tag = singer + " - " + album
      album_infos = network.get_album_infos(url=url)
    except:
      album_infos = []
    
    if not album_infos:
      try:
        # Try search then
        url_tag = urllib.parse.quote(tag)
        album_infos = network.get_album_infos(name=url_tag) # get album by search
      except IndexError as e:
        new_tags.append((tag,uid))
        print("❌ ", tag)
      
    if not album_infos:
      new_tags.append((tag,uid))
    else:
      print("✅ ", tag)
      album_infos["Id"] = uid
      df = pd.concat([df, pd.DataFrame([album_infos])], axis=0)
    # except IndexError:
    # except (IndexError, AttributeError):

  tags = new_tags
  iter += 1

✅  Noname - Room 25
✅  Rolo Tomassi - Time Will Die and Love Will Bury It
✅  Architects - Holy Hell
✅  Robyn - Honey
✅  Yves Tumor - Safe In The Hands Of Love
✅  Kacey Musgraves - Golden Hour
✅  Christine and the Queens - Chris
✅  Rival Consoles - Persona
✅  Jeff Rosenstock - POST-
✅  Idles - Joy As an Act of Resistance
✅  Behemoth - I Loved You at Your Darkest
✅  Marianne Faithfull - Negative Capability
✅  Janelle Monáe - Dirty Computer
✅  Car Seat Headrest - Twin Fantasy
✅  Hookworms - Microshift
✅  Alela Diane - Cusp
✅  Objekt - Cocoon Crush
✅  Daughters - You Won't Get What You Want
✅  Mitski - Be the Cowboy
✅  serpentwithfeet - Soil
✅  U.S. Girls - In a Poem Unlimited
✅  Young Fathers - Cocoa Sugar
✅  Amen Dunes - Freedom
✅  Kali Uchis - Isolation
✅  Bruce Springsteen - Springsteen on Broadway
✅  SOPHIE - Oil Of Every Pearl's Un-Insides
✅  Ashley Monroe - Sparrow
✅  Tracyanne & Danny - Tracyanne & Danny
✅  Hot Snakes - Jericho Sirens
✅  Angélique Kidjo - Remain In Light


Rate-limit detected. Restarting browser.


✅  Kamasi Washington - Heaven & Earth
✅  Mélissa Laveaux - Radyo Siwèl
✅  Low - Double Negative
✅  Amanda Shires - To the Sunset
✅  Mount Eerie - (After) [Live]
✅  Pusha T - DAYTONA
✅  Jonny Greenwood - Phantom Thread [Original Motion Picture Soundtrack]
✅  Earl Sweatshirt - Some Rap Songs
✅  Screaming Females - All at Once
✅  DJ Koze - Knock Knock
✅  Alva Noto - Glass
✅  BROCKHAMPTON - Iridescence
✅  Deafheaven - Ordinary Corrupt Human Love
✅  Marie Davidson - Working Class Woman
✅  Ezra Furman - Transangelic Exodus
✅  Bettye LaVette - Things Have Changed
✅  Let's Eat Grandma - I'm All Ears
✅  Neko Case - Hell-On
✅  Pistol Annies - Interstate Gospel
✅  Sarah Davachi - Gave in Rest
✅  Laurie Anderson - Landfall
✅  Virginia Wing - Ecstatic Arrow
✅  Travis Scott - Astroworld
✅  Troye Sivan - Bloom
✅  Spiritualized - And Nothing Hurt
✅  Jeff Tweedy - Warm
✅  No Age - Snares Like a Haircut
✅  Richard Swift - The Hex
✅  Kids See Ghosts - Kids See Ghosts


Rate-limit detected. Restarting browser.


✅  Lonnie Holley - MITH
✅  Willie Nelson - Last Man Standing
✅  The Necks - Body
✅  CupcakKe - Ephorize
✅  Rolling Blackouts Coastal Fever - Hope Downs
✅  Proc Fiskal - Insula
✅  Cardi B - Invasion of Privacy
✅  Jon Hopkins - Singularity
✅  Sleep - The Sciences
✅  Stephen Malkmus & the Jicks - Sparkle Hard
✅  The Breeders - All Nerve
✅  Jon Hassell - Listening to Pictures (Pentimento Volume I)
✅  Miya Folick - Premonitions
✅  Blood Orange - Negro Swan
✅  Eleanor Friedberger - Rebound
✅  Chilly Gonzales - Solo Piano III
✅  Laurel Halo - Raw Silk Uncut Wood
✅  Anna Calvi - Hunter
✅  Jean Grae - Everything's Fine
✅  Daniel Avery - Song for Alpha
✅  Ty Segall - Freedom's Goblin
✅  Alexander Tucker - Don't Look Away
✅  Jlin - Autobiography
✅  Thou - Magus
✅  Julia Holter - Aviary
✅  Lucrecia Dalt - Anticlines
✅  Elvis Costello & the Imposters - Look Now
✅  Fucked Up - Dose Your Dreams
✅  nao - Saturn


...

✅  Broken Bells - Into the Blue
✅  Jack White - Fear of the Dawn
✅  Tate McRae - I Used to Think I Could Fly
✅  The Waterboys - All Souls Hill
✅  Tegan and Sara - Crybaby
✅  Kasabian - The Alchemist's Euphoria
✅  Lupe Fiasco - Drill Music in Zion
✅  Bruce Springsteen - Only the Strong Survive
✅  Cowboy Junkies - Songs of the Recollection
✅  Joey Bada$$ - 2000
✅  Why Bonnie - 90 in November
✅  Liam Gallagher - C'mon You Know
✅  Flohio - Out Of Heart
✅  Lucius - Second Nature
✅  Bonobo - Fragments
❌  Sonic Youth - In/Out/In
✅  The Mountain Goats - Bleed Out
✅  Avril Lavigne - Love Sux
✅  Christine and the Queens - Redcar les adorable étoiles
✅  Mount Kimbie - MK 3.5: Die Cuts | City Planning
✅  Foals - Life Is Yours
✅  Los Bitchos - Let the Festivities Begin!
✅  Silverbacks - Archive Material
✅  Pillow Queens - Leave the Light On
✅  Arcade Fire - WE
✅  Pinegrove - 11:11
✅  Pixies - Doggerel
✅  The Black Keys - Dropout Boogie
✅  Drake - HONESTLY, NEVERMIND
✅  Hatchie - Giving the World Aw

Rate-limit detected. Restarting browser.


✅  Poliça - Madness
✅  Sloan - Steady
✅  Bastille - Give Me the Future
✅  Mabel - About Last Night...
✅  The Wombats - Fix Yourself, Not the World
✅  Preoccupations - Arrangements
✅  Girlpool - Forgiveness
✅  Interpol - The Other Side of Make-Believe
✅  Sinead O'Brien - Time Bend And Break The Bower
✅  Metronomy - Small World
✅  Eels - Extreme Witchcraft
✅  Flume - Palaces
✅  King Gizzard & the Lizard Wizard - Omnium Gatherum
✅  Pale Waves - Unwanted
✅  Muse - Will of the People
✅  Years & Years - Night Call
✅  Red Hot Chili Peppers - Unlimited Love
✅  Cordae - From A Birds Eye View
✅  Shearwater - The Great Awakening
✅  Miles Kane - Change the Show
✅  Porcupine Tree - Closure/Continuation
✅  Maya Hawke - MOSS
✅  Mattiel - Georgia Gothic
✅  Banks - Serpentina
✅  Rex Orange County - Who Cares?
✅  Fred again.. - Actual Life 3 (January 1 - September 9 2022)
✅  Superorganism - World Wide Pop
✅  BlackPink - Born Pink


Rate-limit detected. Restarting browser.


❌  Marcus Mumford - (self-titled)
✅  Red Hot Chili Peppers - Return of the Dream Canteen
✅  Aitch - Close to Home
✅  Future - I NEVER LIKED YOU
✅  Yungblud - Yungblud
✅  Post Malone - Twelve Carat Toothache
✅  MØ - Motordrome
❌  St. Paul & the Broken Bones - The Alien Coast
✅  Roger Eno - The Turning Year
✅  Lauv - All 4 Nothing
✅  Neneh Cherry - The Versions
✅  Mura Masa - Demon Time
✅  Lil Baby - It's Only Me
✅  Bloc Party - Alpha Games
✅  Turnover - Myself in the Way
✅  Whitney - SPARK
✅  Two Door Cinema Club - Keep on Smiling
✅  Drake - Her Loss
✅  Julian Lennon - Jude
✅  Calvin Harris - Funk Wav Bounces, Vol. 2
✅  2 Chainz - Dope Don't Sell Itself
✅  Machine Gun Kelly - Mainstream Sellout
✅  Jack Harlow - Come Home the Kids Miss You
✅  Jeff Beck - 18


In [88]:
xdf = df.reset_index()
xdf = df[cols]
xdf.reset_index()
xdf.describe()

,Id,Name,Artist,Released,RYM Rating,Genres,Descriptors,Language,Track listing,Colorscheme,Languages,Ranked
count,2257,2257,2257,2257,2251,2257,2257,1735,2257,2257,77,1800
unique,2254,2243,1761,350,2222,2160,1950,13,2229,2171,30,1800
top,arca2021kiciii,Original Motion Picture Soundtrack,Nas,2 March 2018,3.37\n \n/ 5.00.5\n\n ...,"Trap, Southern Hip Hop\nPop Rap",,English,[],"[#7a7a7a, #cccccc]","English, Spanish","#58 for 2018, #8,307 overall"
freq,3,3,6,22,3,8,166,1718,26,75,25,1


In [89]:
xdf["Language"] = [xdf.iloc[i]["Languages"] if pd.isna(xdf.iloc[i]["Language"]) else xdf.iloc[i]["Language"] for i in range(len(xdf.index))]

/opt/homebrew/anaconda3/envs/edav/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [90]:
xdf = xdf[['Id', 'Name', 'Artist', 'Released', 'RYM Rating', 'Genres', 'Descriptors',
       'Language', 'Track listing', 'Colorscheme','Ranked']]
xdf.describe()

,Id,Name,Artist,Released,RYM Rating,Genres,Descriptors,Language,Track listing,Colorscheme,Ranked
count,2257,2257,2257,2257,2251,2257,2257,1812,2257,2257,1800
unique,2254,2243,1761,350,2222,2160,1950,43,2229,2171,1800
top,arca2021kiciii,Original Motion Picture Soundtrack,Nas,2 March 2018,3.37\n \n/ 5.00.5\n\n ...,"Trap, Southern Hip Hop\nPop Rap",,English,[],"[#7a7a7a, #cccccc]","#58 for 2018, #8,307 overall"
freq,3,3,6,22,3,8,166,1718,26,75,1


In [91]:
import re
ratings = []
num_ratings = []
for i in xdf["RYM Rating"]:
  if isinstance(i, float):
    ratings.append("")
    num_ratings.append("")
  else:
    ratings.append(float(i.rsplit('\n')[0]))
    num_ = i.rsplit('\n')[-1]
    num = int(re.findall(r"\d*,*\d+",num_)[0].replace(",",""))
    num_ratings.append(num)

In [92]:
xdf["RYM Rating"] = ratings
xdf["Num_of_RYM_Ratings"] = num_ratings

In [93]:
xdf

,Id,Name,Artist,Released,RYM Rating,Genres,Descriptors,Language,Track listing,Colorscheme,Ranked,Num_of_RYM_Ratings
0,noname2018room25,Room 25,Noname,14 September 2018,3.59,"Jazz Rap, Conscious Hip Hop\nNeo-Soul, Jazz Po...","female vocals, introspective, conscious, po...",English,"[Self, Blaxploitation, Prayer Song, Window, Do...","[#cccccc, #afaeaa, #c9c9c8, #9c9c9a, #b7b7b7, ...","#58 for 2018, #8,307 overall",7439
0,rolotomassi2018timyit,Time Will Die and Love Will Bury It,Rolo Tomassi,2 March 2018,3.53,"Post-Hardcore, Mathcore\nPost-Rock, Blackgaze,...","female vocals, anxious, ethereal, progressi...",English,"[Towards Dawn, Aftermath, Rituals, The Hollow ...","[#b5b4b4, #9b9ba3, #6a6979, #9d9d9f, #7c7b89, ...",#167 for 2018,2307
0,architects2018holell,Holy Hell,Architects,9 November 2018,3.47,"Metalcore\nDjent, Post-Hardcore","male vocals, death, heavy, passionate, ene...",English,"[Death Is Not Defeat, Hereafter, Mortal After ...","[#7a7a7a, #cccccc]",#240 for 2018,1408
0,robyn2018honney,Honey,Robyn,26 October 2018,3.44,"Dance-Pop, Electropop, House\nDeep House, Ambi...","female vocals, atmospheric, sensual, rhythm...",English,"[Missing U, Human Being, Because It's in the M...","[#da978f, #9a4739, #bb3623, #bba597, #7a5d3d, ...",#283 for 2018,3759
0,yvestumor2018safove,Safe in the Hands of Love,Yves Tumor,5 September 2018,3.61,"Neo-Psychedelia, Post-Industrial, Art Pop\nHyp...","psychedelic, eclectic, surreal, atmospheric...",English,"[Faith in Nothing Except in Salvation, Economy...","[#8d989c, #214d58, #6ea3d5, #29767c, #407c96]","#45 for 2018, #6,920 overall",9801
...,...,...,...,...,...,...,...,...,...,...,...,...
0,calvinharris2022funol2,Funk Wav Bounces Vol. 2,Calvin Harris,5 August 2022,2.51,"Dance-Pop, Contemporary R&B, Disco\nSynth Funk...","summer, female vocals, male vocals, party, ...",English,"[Intro, New Money, Potion, Woman of the Year, ...","[#96aeb8, #72a5c2, #68a8cb, #217486, #899297, ...","#1,719 for 2022",1018
0,2chainz2022dopelf,Dope Don’t Sell Itself,2 Chainz,4 February 2022,2.41,"Trap, Southern Hip Hop, Pop Rap","male vocals, drugs, rhythmic, urban, hedon...",English,"[Bet It Back, Pop Music, Kingpen Ghostwriter, ...","[#7a7a7a, #c1c3c0, #e1412b, #7d0e17]","#1,961 for 2022",637
0,machinegunkelly2022maiout,Mainstream Sellout,Machine Gun Kelly,25 March 2022,1.16,"Pop Punk, Alternative Rock\nEmo Rap, Pop Rock,...","male vocals, self-hatred, pessimistic, drug...",English,"[Born With Horns, God Save Me, Maybe, Drug Dea...","[#b8bcbe, #c4c4c8, #937088, #464f56, #dd95bb, ...",NaN,2511
0,jackharlow2022comyou,Come Home the Kids Miss You,Jack Harlow,6 May 2022,1.69,"Pop Rap, Southern Hip Hop\nTrap, Contemporary R&B","male vocals, sexual, hedonistic, sampling, ...",English,"[Talk of the Town, Young Harleezy, I'd Do Anyt...","[#b9b1b0, #918b8b, #cfc1b8, #b1b2ad, #797069]",NaN,2475


In [94]:
xdf.to_csv("rym_add.csv")